In [4]:
import operator
import numpy as np
import matplotlib.pyplot as plt
import nltk
import sys
import csv
import itertools
# from utils import *
from datetime import datetime


In [5]:
nltk.download("book")

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     /Users/pallavitaneja/nltk_data...
[nltk_data]    |   Package conll2002 is 

True

In [12]:
import pandas as pd
import nltk
import itertools
vocab_size = 8000
unknwn = "<unk>"
start_tag = "START_TAG"
end_tag = "END_TAG"
df = pd.read_csv('/Users/pallavitaneja/Downloads/2018_10-reddit000000000000-1.csv')
comments = df[['body']]
comments = comments[comments.body != '[removed]']
comments = comments[comments.body != '[deleted]']
comments = comments[1:50000]
# comments = comments.to_string()
# print(comments)
# for row in comments.iterrows():
#     print(str(row[1]['body']))
sentences = itertools.chain(*[nltk.sent_tokenize(str(x[1]['body']).lower()) for x in comments.iterrows()])
# print(sentences)
# sentences = ["%s %s %s" % (start_tag, x, end_tag) for x in sentences]
sentences = [start_tag +" " + x + " "+ end_tag for x in sentences]
# for sent in sentences:
#     print(type(sent))
# print(type(sentences))
print("Parsed " + str(len(sentences)) +" sentences." )
tokens = [nltk.word_tokenize(sent) for sent in sentences]
# # print(tokens)
freq_of_word = nltk.FreqDist(itertools.chain(*tokens))
print(type(freq_of_word))
print("Found "+ str(len(freq_of_word.items())) +" unique s tokens.")
bag_of_words = freq_of_word.most_common(vocab_size - 1)
idx_to_w = [x[0] for x in bag_of_words]
idx_to_w.append(unknwn)
w_idx = {}
w_idx = dict([(w,i) for i,w in enumerate(idx_to_w)])
print(len(bag_of_words))
print("least frequent  is " + str(bag_of_words[-1][0])+ " with frequency"+ str(bag_of_words[-1][1]))
for each, sent in enumerate(tokens):
    tokens[each] = [w if w in w_idx else unknwn for w in sent]
print("Normal Sentence:" + str(sentences[0]))
print("Tokenized sentence:" + str(tokens[0]))
#     print(i)
# print(bag_of_s)

Parsed 113101 sentences.
<class 'nltk.probability.FreqDist'>
Found 71818 unique s tokens.
7999
least frequent  is 51 with frequency10
Normal Sentence:START_TAG there are people like myself who would like to live more simply but are overwhelmed by all the change would entail, who like to lurk. END_TAG
Tokenized sentence:['START_TAG', 'there', 'are', 'people', 'like', 'myself', 'who', 'would', 'like', 'to', 'live', 'more', 'simply', 'but', 'are', '<unk>', 'by', 'all', 'the', 'change', 'would', '<unk>', ',', 'who', 'like', 'to', '<unk>', '.', 'END_TAG']


In [13]:
import numpy as np
XTrain = []
for sent in tokens:
    x = []
    for w in sent[:-1]:
        x.append(w_idx[w])
    XTrain.append(x)
YTrain = []
for sent in tokens:
    y = []
    for w in sent[1:]:
        y.append(w_idx[w])
    YTrain.append(y)
XTrain = np.asarray(XTrain)
YTrain = np.asarray(YTrain)
print(XTrain[0])
print(YTrain[0])

[0, 55, 32, 67, 43, 400, 96, 51, 43, 6, 314, 66, 781, 25, 32, 7999, 83, 53, 3, 355, 51, 7999, 4, 96, 43, 6, 7999, 2]
[55, 32, 67, 43, 400, 96, 51, 43, 6, 314, 66, 781, 25, 32, 7999, 83, 53, 3, 355, 51, 7999, 4, 96, 43, 6, 7999, 2, 1]


In [14]:
x_example, y_example = XTrain[17], YTrain[17]
for x in x_example :
    print(str(idx_to_w[x]))
print("\n")
for y in y_example:
    print(str(idx_to_w[y]))

START_TAG
indians
are
breaking
my
heart


indians
are
breaking
my
heart
END_TAG


In [15]:
class np_RNN:
    
    def __init__(self, dim_of_word, dim_of_hl=100, back_prop_param=4):
        # Assign instance variables
        # Randomly initialize shared parameters U, V, W as 
        self.dim_of_word = dim_of_word
        self.dim_of_hl = dim_of_hl                                       
        self.back_prop_param = back_prop_param
        self.U = np.random.uniform(-np.sqrt(1./dim_of_word), np.sqrt(1./dim_of_word), (dim_of_hl, dim_of_word))
        self.V = np.random.uniform(-np.sqrt(1./dim_of_hl), np.sqrt(1./dim_of_hl), (dim_of_word, dim_of_hl))
        self.W = np.random.uniform(-np.sqrt(1./dim_of_hl), np.sqrt(1./dim_of_hl), (dim_of_hl, dim_of_hl))

In [16]:
def softmax(x):
    xt = np.exp(x - np.max(x))
    return xt / np.sum(xt)
def forward_propagation(self, x):
    T = len(x)
    # initialize start state to zero
    s = np.zeros((T + 1, self.dim_of_hl))
    s[-1] = np.zeros(self.dim_of_hl)
    o = np.zeros((T, self.dim_of_word))
    for t in np.arange(T):
        s[t] = np.tanh(self.U[:,x[t]] + self.W.dot(s[t-1]))
        o[t] = softmax(self.V.dot(s[t]))
    return [o, s]

np_RNN.forward_propagation = forward_propagation                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [17]:
# predict function to perform forward propagation 
def predict(self, x):
    o, s = self.forward_propagation(x)
    return np.argmax(o, axis=1)
#return index of the highest score
np_RNN.predict = predict

In [18]:
import numpy as np
np.random.seed(10)
model = np_RNN(vocab_size)
o, s = model.forward_propagation(XTrain[10])
print(o)
print(o.shape)
predictions = model.predict(XTrain[10])
print(predictions.shape)
print(predictions)

[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012399 0.00012521 0.00012424 ... 0.00012493 0.00012543 0.00012386]]
(2, 8000)
(2,)
[1284 4855]


In [37]:
def cross_entropy_loss(self, x, y):
    Loss = 0
    for i in np.arange(len(y)):
        print(y[i])
        
        o, s = self.forward_propagation(x[i])
        print(o.shape)
        print(o)
        #C = correct predictions
        C = o[np.arange(len(y[i])), y[i]]
        #average loss = logC
        Loss += -1 * np.sum(np.log(C))
        print(C)
        print(C.shape)
    return Loss

def calculate_loss(self, x, y):
    N = np.sum((len(y_i) for y_i in y))
    return self.cross_entropy_loss(x,y)/N

np_RNN.cross_entropy_loss = cross_entropy_loss
np_RNN.calculate_loss = calculate_loss

In [39]:
#1000 examples 
print("Expected Loss for random predictions: " +  str(np.log(vocab_size)))
print("Actual loss:" + str(model.calculate_loss(XTrain[:1000], YTrain[:1000])))

Expected Loss for random predictions: 8.987196820661973
[55, 32, 67, 43, 400, 96, 51, 43, 6, 314, 66, 781, 25, 32, 7999, 83, 53, 3, 355, 51, 7999, 4, 96, 43, 6, 7999, 2, 1]
(28, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012481 0.00012597 0.00012455 ... 0.00012542 0.00012455 0.00012494]
 [0.00012593 0.00012551 0.00012474 ... 0.00012605 0.00012474 0.00012592]
 ...
 [0.00012459 0.00012506 0.00012493 ... 0.00012493 0.00012481 0.00012524]
 [0.00012545 0.0001248  0.00012495 ... 0.00012489 0.0001261  0.00012509]
 [0.0001248  0.00012424 0.00012488 ... 0.00012477 0.00012487 0.00012483]]
[0.00012564 0.00012492 0.00012516 0.00012561 0.00012454 0.00012512
 0.00012537 0.00012563 0.00012424 0.00012506 0.0001241  0.00012623
 0.0001249  0.00012534 0.00012521 0.00012478 0.00012483 0.00012488
 0.00012576 0.00012452 0.00012451 0.00012515 0.00012506 0.00012522
 0.0001248  0.00012524 0.00012495 0.00012424]
(28,)
[26, 20, 1]
(3, 8000)
[[0.00012408 0.0001244  0.0001

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.


(12, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012469 0.00012544 0.0001247  ... 0.00012475 0.00012431 0.00012368]
 [0.00012586 0.00012523 0.00012488 ... 0.00012499 0.00012451 0.00012539]
 ...
 [0.00012451 0.00012401 0.000125   ... 0.00012558 0.00012556 0.00012433]
 [0.00012547 0.00012465 0.00012497 ... 0.0001249  0.00012519 0.00012498]
 [0.00012511 0.00012447 0.00012487 ... 0.00012438 0.00012468 0.00012507]]
[0.00012405 0.00012452 0.00012507 0.00012512 0.00012483 0.00012516
 0.00012503 0.00012522 0.00012475 0.0001249  0.00012497 0.00012447]
(12,)
[164, 23, 166, 278, 13, 341, 249, 2344, 4740, 2991, 14, 538, 46, 80, 194, 17, 404, 2, 1]
(19, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.000124   0.00012564 0.00012523 ... 0.0001253  0.00012433 0.00012551]
 [0.00012484 0.00012384 0.00012453 ... 0.00012502 0.00012537 0.00012539]
 ...
 [0.00012491 0.00012525 0.00012594 ... 0.00012459 0.0001244  0.00012517]
 [0.000

(19, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012499 0.00012508 0.00012456 ... 0.0001249  0.0001246  0.00012438]
 [0.00012487 0.00012533 0.00012465 ... 0.00012532 0.00012558 0.00012559]
 ...
 [0.00012501 0.00012415 0.0001242  ... 0.00012605 0.00012495 0.0001252 ]
 [0.00012553 0.00012428 0.0001256  ... 0.00012443 0.00012427 0.00012421]
 [0.00012517 0.00012529 0.00012476 ... 0.00012465 0.00012584 0.00012477]]
[0.00012519 0.00012498 0.0001255  0.00012418 0.00012593 0.00012502
 0.0001247  0.00012501 0.00012459 0.00012523 0.00012447 0.00012486
 0.00012553 0.00012515 0.00012547 0.00012516 0.00012514 0.0001253
 0.00012529]
(19,)
[78, 104, 60, 3293, 3, 7999, 228, 16, 1]
(9, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012461 0.00012578 0.00012496 ... 0.00012488 0.00012469 0.00012533]
 [0.00012508 0.00012595 0.00012473 ... 0.00012512 0.00012554 0.00012499]
 ...
 [0.0001251  0.00012504 0.0001252  ... 0.00012503 0

(2, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012466 0.00012542 0.00012408 ... 0.00012486 0.00012593 0.00012505]]
[0.00012601 0.00012542]
(2,)
[203, 810, 703, 2635, 16, 1]
(6, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012461 0.00012575 0.00012464 ... 0.00012533 0.0001249  0.00012528]
 [0.00012463 0.00012529 0.00012459 ... 0.00012549 0.00012468 0.00012458]
 [0.00012509 0.00012481 0.00012565 ... 0.00012506 0.00012394 0.00012534]
 [0.00012567 0.00012433 0.00012505 ... 0.00012549 0.00012466 0.00012436]
 [0.00012496 0.00012466 0.00012428 ... 0.00012492 0.00012443 0.00012479]]
[0.00012453 0.00012522 0.00012459 0.00012567 0.00012545 0.00012466]
(6,)
[33, 44, 1023, 18, 71, 7, 2757, 155, 44, 13, 105, 2, 1]
(13, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012488 0.00012412 0.00012515 ... 0.00012353 0.00012552 0.00012514]
 [0.00012507 0.00012532 0.00012384 ... 0.00012424 

...

(10, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012463 0.00012469 0.00012536 ... 0.00012556 0.00012454 0.00012494]
 [0.00012512 0.00012522 0.00012521 ... 0.00012559 0.00012414 0.00012669]
 ...
 [0.00012535 0.00012484 0.00012515 ... 0.000126   0.00012491 0.0001253 ]
 [0.00012484 0.00012483 0.00012404 ... 0.00012508 0.0001248  0.00012497]
 [0.00012344 0.00012514 0.00012572 ... 0.00012509 0.00012429 0.00012542]]
[0.00012594 0.00012519 0.00012554 0.00012573 0.00012516 0.00012401
 0.00012481 0.00012574 0.00012441 0.00012514]
(10,)
[42, 49, 33, 7999, 1933, 4572, 2, 1]
(8, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012469 0.00012544 0.0001247  ... 0.00012475 0.00012431 0.00012368]
 [0.00012571 0.0001253  0.00012512 ... 0.00012534 0.0001247  0.00012568]
 ...
 [0.00012495 0.00012515 0.00012443 ... 0.00012434 0.00012502 0.00012511]
 [0.0001258  0.00012535 0.00012571 ... 0.00012513 0.00012485 0.0001254 ]
 [0.00012

(2, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012518 0.00012541 0.000125   ... 0.00012503 0.00012555 0.00012548]]
[0.00012508 0.00012541]
(2,)
[106, 4269, 76, 10, 87, 7908, 130, 6, 1291, 93, 99, 4, 1291, 2, 1]
(15, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012517 0.00012535 0.00012411 ... 0.00012472 0.00012458 0.00012561]
 [0.00012535 0.00012436 0.00012555 ... 0.00012475 0.00012545 0.00012473]
 ...
 [0.0001255  0.00012459 0.00012511 ... 0.00012662 0.00012577 0.0001247 ]
 [0.00012371 0.00012478 0.00012537 ... 0.00012345 0.00012511 0.000126  ]
 [0.00012574 0.0001245  0.00012479 ... 0.00012477 0.00012448 0.00012523]]
[0.0001245  0.00012448 0.00012427 0.00012476 0.0001249  0.00012303
 0.00012527 0.00012603 0.00012501 0.00012607 0.00012469 0.00012542
 0.00012573 0.00012537 0.0001245 ]
(15,)
[5, 18, 148, 115, 278, 7999, 4, 5, 1162, 18, 71, 328, 12, 768, 14, 7, 230, 84, 1]
(19, 8000)
[[0.00012408 0.0001244  

[0.00012501 0.00012548 0.00012488 0.00012534 0.00012535 0.00012449
 0.00012519 0.00012464 0.00012535 0.00012515 0.00012525 0.00012457
 0.00012484 0.00012507 0.00012516 0.00012547 0.00012587 0.00012493
 0.00012555 0.00012312 0.00012442 0.00012472 0.00012513 0.00012402
 0.00012432 0.00012464 0.0001251  0.00012438 0.00012461]
(29,)
[39, 241, 80, 7999, 7999, 14, 38, 7999, 15, 7, 1700, 7999, 1212, 2, 1]
(15, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012462 0.00012555 0.00012549 ... 0.00012489 0.00012504 0.00012457]
 [0.00012594 0.00012453 0.00012457 ... 0.00012517 0.00012529 0.00012505]
 ...
 [0.00012537 0.00012484 0.0001252  ... 0.0001247  0.00012521 0.00012573]
 [0.0001242  0.00012444 0.00012496 ... 0.00012411 0.00012435 0.00012518]
 [0.0001254  0.00012466 0.00012472 ... 0.00012467 0.00012501 0.00012525]]
[0.00012439 0.00012413 0.00012507 0.00012481 0.00012566 0.00012514
 0.00012534 0.00012445 0.00012469 0.00012455 0.00012522 0.00012502
 0.000124

(20, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012497 0.00012536 0.00012416 ... 0.00012475 0.00012528 0.00012521]
 [0.00012393 0.00012512 0.00012469 ... 0.00012551 0.00012392 0.00012417]
 ...
 [0.00012543 0.00012523 0.000125   ... 0.00012539 0.00012602 0.00012651]
 [0.00012499 0.00012416 0.00012522 ... 0.00012518 0.00012439 0.00012574]
 [0.00012525 0.00012415 0.00012503 ... 0.0001246  0.00012505 0.00012462]]
[0.00012456 0.00012543 0.0001247  0.00012463 0.00012469 0.00012469
 0.00012529 0.00012657 0.00012407 0.00012463 0.0001256  0.00012448
 0.00012613 0.00012441 0.00012569 0.00012466 0.00012515 0.0001245
 0.00012522 0.00012415]
(20,)
[5, 51, 381, 30, 3, 7999, 11, 38, 5153, 4, 25, 5, 76, 9, 51, 28, 5360, 41, 7, 491, 518, 2, 1]
(23, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.00012499 0.00012508 0.00012456 ... 0.0001249  0.0001246  0.00012438]
 [0.00012611 0.00012561 0.00012594 ... 0.00012496 0.00012578 0.0

(41, 8000)
[[0.00012408 0.0001244  0.00012603 ... 0.00012515 0.00012488 0.00012508]
 [0.0001247  0.00012573 0.00012384 ... 0.00012446 0.00012475 0.00012594]
 [0.00012514 0.00012421 0.00012485 ... 0.00012493 0.00012475 0.00012494]
 ...
 [0.00012664 0.00012502 0.00012433 ... 0.00012489 0.00012645 0.00012504]
 [0.00012426 0.00012519 0.00012493 ... 0.0001245  0.00012418 0.00012492]
 [0.00012504 0.00012405 0.00012475 ... 0.00012469 0.00012503 0.00012496]]
[0.00012485 0.00012571 0.00012468 0.00012564 0.00012609 0.00012557
 0.00012499 0.00012483 0.00012394 0.00012429 0.00012471 0.00012505
 0.00012502 0.00012419 0.00012461 0.00012454 0.00012436 0.00012487
 0.0001247  0.00012469 0.00012419 0.00012448 0.00012407 0.00012435
 0.00012456 0.00012435 0.00012545 0.00012546 0.00012432 0.00012553
 0.00012461 0.00012578 0.00012556 0.00012559 0.00012432 0.00012523
 0.00012485 0.00012557 0.00012515 0.00012493 0.00012405]
(41,)
[5, 1162, 18, 71, 7999, 38, 3753, 4, 5, 76, 9, 18, 57, 1020, 4, 10, 1162, 18, 71

In [ ]:
def bptt(self, x, y):
    T = len(y)
    # Perform forward propagation
    o, s = self.forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(self.U.shape)
    dLdV = np.zeros(self.V.shape)
    dLdW = np.zeros(self.W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation
        delta_t = self.V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-self.bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

np_RNN.bptt = bptt

In [ ]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = model.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print("Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape)))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = model.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = model.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error > error_threshold:
                print("Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix))
                print("+h Loss: %f" % gradplus)
                print("-h Loss: %f" % gradminus)
                print("Estimated_gradient: %f" % estimated_gradient)
                print("Backpropagation gradient: %f" % backprop_gradient)
                print("Relative Error: %f" % relative_error)
                return 
            it.iternext()
        print("Gradient check for parameter %s passed." % (pname))

np_RNN.gradient_check = gradient_check

# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = np_RNN(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])

In [ ]:
# Performs one step of SGD.
def numpy_sdg_step(self, x, y, learning_rate):
    # Calculate the gradients
    dLdU, dLdV, dLdW = self.bptt(x, y)
    # Change parameters according to gradients and learning rate
    self.U -= learning_rate * dLdU
    self.V -= learning_rate * dLdV
    self.W -= learning_rate * dLdW

np_RNN.sgd_step = numpy_sdg_step

In [ ]:
# Outer SGD Loop
# - model: The RNN model instance
# - X_train: The training data set
# - y_train: The training data labels
# - learning_rate: Initial learning rate for SGD
# - nepoch: Number of times to iterate through the complete dataset
# - evaluate_loss_after: Evaluate the loss after this many epochs
def train_with_sgd(model, X_train, y_train, learning_rate=0.05, nepoch=100, evaluate_loss_after=5):
    # We keep track of the losses so we can plot them later
    losses = []
    num_examples_seen = 0
    for epoch in range(nepoch):
        # Optionally evaluate the loss
        if (epoch % evaluate_loss_after == 0):
            loss = model.calculate_loss(X_train, y_train)
            losses.append((num_examples_seen, loss))
            time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print("%s: Loss after num_examples_seen=%d epoch=%d: %f" % (time, num_examples_seen, epoch, loss))
            # Adjust the learning rate if loss increases
            if (len(losses) > 1 and losses[-1][1] > losses[-2][1]):
                learning_rate = learning_rate * 0.5  
                print("Setting learning rate to %f" % learning_rate)
            sys.stdout.flush()
        # For each training example...
        for i in range(len(y_train)):
            # One SGD step
            model.sgd_step(X_train[i], y_train[i], learning_rate)
            num_examples_seen += 1

In [ ]:
np.random.seed(10)
model = np_RNN(vocab_size)
%timeit model.sgd_step(XTrain[10], YTrain[10], 0.005)

In [ ]:
np.random.seed(10)
# Train on a small subset of the data to see what happens
model = np_RNN(vocab_size)
losses = train_with_sgd(model, XTrain[:1000], YTrain[:1000], nepoch=10, evaluate_loss_after=1)

In [ ]:
def generate_sentence(model):
    # We start the sentence with the start token
    new_sentence = [w_idx[start_tag]]
    # Repeat until we get an end token
    
    while not new_sentence[-1] == w_idx[end_tag]:
        next__probs = model.forward_propagation(new_sentence)[0]
#         print(next__probs.shape)
        sampled_ = w_idx[unknwn]
        # We don't want to sample unknown s
        while sampled_ == w_idx[unknwn]:
            samples = np.random.multinomial(1, next__probs[-1])
            sampled_ = np.argmax(samples)
        new_sentence.append(sampled_)
    sentence_str = [idx_to_w[x] for x in new_sentence[1:-1]]
    return sentence_str

num_sentences = 10
senten_min_length = 7

for i in range(num_sentences):
    sent = []
    # We want long sentences, not sentences with one or two s
    while len(sent) < senten_min_length:
        sent = generate_sentence(model)
    print(" ".join(sent))